In [5]:
import ee
import geemap


# Initialize the Earth Engine Python API
ee.Initialize()

# Initialize the Map
Map = geemap.Map()



# Load your Asset ROI by specifying the path to your Asset
asset_roi = ee.FeatureCollection('projects/ee-hydroperiod/assets/ROI')  # Replace with the path to your Asset


# Define a region of interest (ROI) based on your Asset ROI.
geometry = asset_roi.geometry()


# Define the date range
start_date = '2021-12-01'
end_date = '2022-02-28'

#Create Sentinal-2 Image Collection from date range with images that have 35% cloud cover or less within roi
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35))



# Get a list of image IDs from the Sentinel-2 Image Collection
image_ids = s2.aggregate_array('system:index')
image_ids_list = image_ids.getInfo()



# Create a new Image Collection by filtering the DYNAMICWORLD collection
dynamic_world_collection = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filter(ee.Filter.inList('system:index', image_ids_list))

# Get a list of images in the Image Collection
image_list = dynamic_world_collection.toList(dynamic_world_collection.size())

# Loop through the list of images and export each image
for i in range(image_list.size().getInfo()):
    image = ee.Image(image_list.get(i))
    image_id = image.id().getInfo()  # Convert to Python string
    date = ee.Date(image.get('system:time_start'))
    date_string = date.format('YYYY_MM_dd').getInfo()  # Convert to Python string

    # Select the 'label' band for each image
    classification = image.select('label')

    # Convert to boolean where label equals 0 or 3
    boolean_classification = classification.eq(0).Or(classification.eq(3))

    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=boolean_classification,
        description=date_string,
        scale=10,
        region=geometry,
        maxPixels=1e12,
        folder='GEE_Hydroperiod',
        formatOptions={
            'cloudOptimized': True
        }
    )
    
    # Start the export task
    task.start()
    
    # Print a message for each export task
    print('Exporting image:', date_string)


Exporting image: 2021_12_02
Exporting image: 2021_12_12
Exporting image: 2021_12_22
Exporting image: 2021_12_22
Exporting image: 2021_12_27
Exporting image: 2021_12_27
Exporting image: 2022_01_01
Exporting image: 2022_01_01
Exporting image: 2022_01_06
Exporting image: 2022_01_06
Exporting image: 2022_01_16
Exporting image: 2022_01_16
Exporting image: 2022_01_26
Exporting image: 2022_01_26
Exporting image: 2022_02_05
Exporting image: 2022_02_05
Exporting image: 2022_02_15
Exporting image: 2022_02_15
